In [1]:
import librosa
import numpy as np
import os
from clip2frame import utils, measure
import numpy as np
import theano
from lasagne import layers
import network_structure as ns
from sklearn.preprocessing import StandardScaler
import pandas as pd


You can find the C code in this temporary file: /var/folders/pw/cbgs3y0j1q3b_zx7_rdh01bc0000gn/T/theano_compilation_error_5jxo1a5n


Exception: Compilation failed (return status=1): In file included from /Users/zhangben/.theano/compiledir_Darwin-18.0.0-x86_64-i386-64bit-i386-3.6.4-64/lazylinker_ext/mod.cpp:1:. In file included from /Users/zhangben/anaconda/include/python3.6m/Python.h:25:. /Users/zhangben/anaconda/bin/../include/c++/v1/stdio.h:108:15: fatal error: 'stdio.h' file not found. #include_next <stdio.h>.               ^~~~~~~~~. 1 error generated.. 

## get feature list of example track, 6 list of mel-spectrogram

In [3]:
def extract_melspec(in_fp, sr, win_size, hop_size, n_mels):
    sig, sr = librosa.core.load(in_fp, sr=sr)
    feat = librosa.feature.melspectrogram(sig, sr=sr,
                                          n_fft=win_size,
                                          hop_length=hop_size,
                                          n_mels=n_mels).T
    feat = np.log(1+10000*feat)
    return feat


in_fp = 'luhu.mp3' #example track, you can replace whatever song you like

sr = 16000
win_size_list = [512, 1024, 2048, 4096, 8192, 16384]
hop_size = 512
n_mels = 128
diff_order = 0

feat_list = []
for win_size in win_size_list:
    feat = extract_melspec(in_fp, sr, win_size, hop_size, n_mels)
    feat_list.append(feat)
# print('Feature shape: {}'.format(feat.shape))

In [4]:
#make sure feature shape are same
for feat in feat_list:
    print('Feature shape: {}'.format(feat.shape))

Feature shape: (9729, 128)
Feature shape: (9729, 128)
Feature shape: (9729, 128)
Feature shape: (9729, 128)
Feature shape: (9729, 128)
Feature shape: (9729, 128)


In [14]:
#base_feat_dir = 'data/data.medleydb/sample_features'

## build pretrained model

In [5]:
# Choosing the function for building the model
build_func = ns.build_fcn_gaussian_multiscale

# The layer for frame-level output: 'gaussian' or 'no-gaussian'
frame_layer_type = 'gaussian'

In [6]:
# Upscale method: 'naive' or 'patching'
upscale_method = 'naive'
threshold_source ='MagnaTagATune'  # 'MedleyDB' or 'MagnaTagATune'

In [7]:
# Files and directories
model_dir = 'data/models'
param_fp = 'data/models/model.20160309_111546.npz'
standardizer_dir = 'data/standardizers'

In [8]:
tag_tr_fp = 'data/data.magnatagatune/tag_list.top188.txt'

In [12]:
# Default setting
upscale_factor = 16  # The total pooling size from all convolution layers
scale_list = [
    "logmelspec10000.16000_512_512_128.0.raw",
    "logmelspec10000.16000_1024_512_128.0.raw",
    "logmelspec10000.16000_2048_512_128.0.raw",
    "logmelspec10000.16000_4096_512_128.0.raw",
    "logmelspec10000.16000_8192_512_128.0.raw",
    "logmelspec10000.16000_16384_512_128.0.raw",
]
# feat_dir_list = [os.path.join(base_feat_dir, scale) for scale in scale_list]
# num_scales = len(scale_list)

In [13]:
# Load tag list
tag188 = utils.read_lines(tag_tr_fp )

In [14]:
# Standardizer dir
std_fp_list = [os.path.join(standardizer_dir,
                            scale.replace('.raw', ''),
                            'scaler.pkl')
               for scale in scale_list]

std_list = [utils.unpickle(std_fp) for std_fp in std_fp_list]
std_list = [StandardScaler() for std_fp in std_list]

/Users/zhangben/anaconda/lib/python3.6/site-packages/sklearn/base.py:311: UserWarning: Trying to unpickle estimator StandardScaler from version pre-0.18 when using version 0.19.1. This might lead to breaking code or invalid results. Use at your own risk.
  UserWarning)


In [15]:
# Building Network
print("Building network...")
num_scales = len(scale_list)
network, input_var_list, nogaussian_layer, gaussian_layer = \
    build_func(num_scales)

Building network...


In [16]:
# Load params
utils.load_model(param_fp, network)

# Get frame output layer
if frame_layer_type == 'gaussian':
    frame_output_layer = gaussian_layer
elif frame_layer_type == 'no-gaussian':
    frame_output_layer = nogaussian_layer

In [17]:
# Make predicting function
frame_prediction = layers.get_output(frame_output_layer, deterministic=True)
func = theano.function(input_var_list, [frame_prediction])

In [18]:
# label threshold
if threshold_source in ['magnatagatune', 'MagnaTagATune']:
    thres_fp = os.path.join(
        model_dir, 'threshold.20160309_111546.with_magnatagatune.npy')
    thresholds_raw = np.load(thres_fp)
    thresholds = thresholds_raw  #[tag_idx_list]
elif threshold_source in ['MedleyDB', 'medleydb']:
    thres_fp = os.path.join(
        model_dir, 'threshold.20160309_111546.with_medleydb.top9.npy')
    thresholds = np.load(thres_fp)

In [19]:
# standardize
feat_list_temp = []
for feat, standardizer in zip(feat_list, std_list): 
    scaler = standardizer
    scaler.fit(feat)
    feat_list_temp.append(scaler.transform(feat)) 
#     feat_list = [standardizer.transform(feat)
#                  for feat, standardizer in zip(feat_list, std_list)]

feat_list = [feat[None, None, :].astype('float32')
             for feat in feat_list_temp]

# Predict and upscale
out_axis = 2
in_axis = 2
prediction = utils.upscale(func, feat_list,
                           upscale_method, upscale_factor,
                           in_axis, out_axis)
# raw_input(123) #for a new song, we need to get this and sort it.
prediction = prediction[0].T

pred_binary = ((prediction-thresholds) > 0).astype(int)
try:
#     anno_all = np.concatenate([anno_all, anno], axis=0)
    pred_all = np.concatenate([pred_all, pred_binary],
                              axis=0)
except:
#     anno_all = anno
    pred_all = pred_binary

In [20]:
#find top 20 tags
sumup  = np.sum(pred_all, axis=0)
top20idx = sumup.argsort()[-20:][::-1]

labels = [tag188[idx] for idx in top20idx]
top20 = pred_all[:,list(top20idx)]

In [23]:
#random check
pd.DataFrame(top20, columns=labels)[900:1000]

,vocal,voice,singing,vocals,male,male vocal,man,male voice,singer,male vocals,pop,guitar,male singer,man singing,folk,slow,english,rock,classical,country
900,1,1,0,0,0,0,0,0,0,0,0,1,0,0,0,1,0,0,0,0
901,1,1,0,0,0,0,0,0,0,0,0,1,0,0,0,1,0,0,0,0
902,1,1,0,0,0,0,0,0,0,0,0,1,0,0,0,1,0,0,0,0
903,1,1,0,0,0,0,0,0,0,0,0,1,0,0,0,1,0,0,0,0
904,1,1,0,0,0,0,0,0,0,0,0,1,0,0,0,1,0,0,0,0
905,1,1,0,0,0,0,0,0,0,0,0,1,0,0,0,1,0,0,0,0
906,1,1,0,0,0,0,0,0,0,0,0,1,0,0,0,1,0,0,0,0
907,1,1,0,0,0,0,0,0,0,0,0,1,0,0,0,1,0,0,0,0
908,1,1,0,0,0,0,0,0,0,0,0,1,0,0,0,1,0,0,0,0
909,1,1,0,0,0,0,0,0,0,0,0,1,0,0,0,1,0,0,0,0
